In [5]:
import numpy as np
import pyrealsense2 as rs

import os
os.environ['QT_QPA_PLATFORM'] = 'xcb'

import cv2
import os
import heapq

import threading
import queue
import time


from video_capture import capture_frames
import math

In [6]:
def calculate_distance(p1, p2):
    """ Calculate the Euclidean distance between two points. """
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [7]:
def cut_region(depth_image,color_image,min_depth = 0,max_depth = 0.8):
  
    min_depth_mm = min_depth * 1000
    max_depth_mm = max_depth * 1000

    # Create a mask for the specified depth range
    mask = np.logical_and(depth_image > min_depth_mm, depth_image < max_depth_mm)

    # Convert mask to uint8
    mask = mask.astype(np.uint8) * 255

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Create an empty mask to draw the contour region
    contour_mask = np.zeros_like(mask)
        # Check if any contours were found
    if contours:
        all_points = np.concatenate(contours)
        hull = cv2.convexHull(all_points)
        # Draw the filled contour on the contour_mask
        cv2.drawContours(contour_mask, [hull], -1, (255), thickness=cv2.FILLED)

        rect = cv2.minAreaRect(hull)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(color_image, [box], 0, (125, 125, 255), 5)
        cv2.drawContours(color_image, [hull], 0, (0, 255, 0), 5)
        

    masked_color_image = cv2.bitwise_and(color_image, color_image, mask=contour_mask)
    
                
    return masked_color_image, hull,box

In [8]:
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

In [9]:
def line_from_points(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    
    # Calculate slope (m)
    if x2 != x1:
        m = (y2 - y1) / (x2 - x1)
    else:
        raise ValueError("The x-coordinates of the points cannot be the same.")
    
    # Calculate y-intercept (b)
    b = y1 - m * x1
    
    return m, b

In [10]:
def get_line_intercection(m1,m2,q1,q2):
    
    x = (q2-q1) / (m1-m2) 
    y = m1*x+q1
    
    x = int(x)
    y = int(y)
    p = tuple([x,y])
    return p

In [11]:
def move_point_on_line(p1, p2, point, distance):
    """
    Move a point along the line defined by p1 and p2 by a certain distance.
    
    Parameters:
    p1 (tuple): First point defining the line (x1, y1).
    p2 (tuple): Second point defining the line (x2, y2).
    point (tuple): The point to move (x, y).
    distance (float): The distance to move along the line.
    
    Returns:
    tuple: New coordinates of the point.
    """
    
    # Convert points to numpy arrays
    p1 = np.array(p1)
    p2 = np.array(p2)
    point = np.array(point)
    
    # Calculate direction vector of the line
    direction = p2 - p1
    
    # Normalize the direction vector
    norm = np.linalg.norm(direction)
    direction_normalized = direction / norm
    
    # Calculate the new point
    new_point = point + direction_normalized * distance
   
    new_x = int(new_point[0])
    new_y = int(new_point[1])
    
    return tuple([new_x,new_y])

In [12]:
def draw_rotated_rectangle(color_image,x1,x2,y1,y2,p_new_1):
                angle = -np.degrees(np.arctan2(y2-y1, x2-x1))
                
                x, y = p_new_1[0], p_new_1[1]

                # Define the width and height of the rectangle
                width = 80
                height = 45

                rotation_matrix = cv2.getRotationMatrix2D((x, y), angle, 1)
                
                corner_1  =[x+width, y-height]
                corner_2 = [x+width, y+height]
                corner_3 = [x-width, y+height]
                corner_4 = [x-width,y-height]
                
                rect_points = np.array([
                corner_1,
                corner_2,
                corner_3,
                corner_4
            ])

                rotated_rect_points = np.dot(rect_points, rotation_matrix[:, :2].T) + rotation_matrix[:, 2]
                
                rotated_rect_points = rotated_rect_points.astype(int)

                rotated_rect_points[:,0] =  rotated_rect_points[:,0]  #+ x
                rotated_rect_points[:,1] =  rotated_rect_points[:,1]  #+ y
                
                cv2.polylines(color_image, [rotated_rect_points], isClosed=True, color=(125, 125, 0), thickness=10)

                return rect_points

In [13]:
def get_corner_points(color_image,box,hull):
                x_corner_1 = 0
                y_corner_1 = 0
                reference_distance_corener_1 = 5000
                
                x_corner_2 = 0
                y_corner_2 = 0
                reference_distance_corener_2 = 5000
                
                x_corner_3 = 0
                y_corner_3 = 0
                reference_distance_corener_3 = 5000
                
                x_corner_4 = 0
                y_corner_4 = 0
                reference_distance_corener_4 = 5000
                
                for j in range(len(hull)):
                    p = tuple(hull[j][0])
                    x = p[0]
                    y = p[1]
                    

                    distance_corner_1 = calculate_distance(box[0], p)
                    distance_corner_2 = calculate_distance(box[1], p)
                    distance_corner_3 = calculate_distance(box[2], p)
                    distance_corner_4 = calculate_distance(box[3], p)
                    
                    if distance_corner_1 < reference_distance_corener_1:
                        x_corner_1 = x
                        y_corner_1 = y
                        reference_distance_corener_1 = distance_corner_1
                        
                    if distance_corner_2 < reference_distance_corener_2:
                        x_corner_2 = x
                        y_corner_2 = y
                        reference_distance_corener_2 = distance_corner_2
                    
                    if distance_corner_3 < reference_distance_corener_3:
                        x_corner_3 = x
                        y_corner_3 = y
                        reference_distance_corener_3 = distance_corner_3
                        
                      
                    if distance_corner_4 < reference_distance_corener_4:
                        x_corner_4 = x
                        y_corner_4 = y
                        reference_distance_corener_4 = distance_corner_4


                cv2.circle(color_image, box[1], 20, (255, 0, 255), 5) 
                cv2.circle(color_image, [x_corner_1,y_corner_1], 20, (0, 125, 255), 5) 
                cv2.circle(color_image, [x_corner_2,y_corner_2], 20, (0, 125, 255), 5) 
                cv2.circle(color_image, [x_corner_3,y_corner_3], 20, (0, 125, 255), 5) 
                cv2.circle(color_image, [x_corner_4,y_corner_4], 20, (0, 125, 255), 5) 
                
                
        
                
                corner_1 = [int(x_corner_1), int(y_corner_1)]
                corner_2 = [x_corner_2, y_corner_2]
                corner_3 = [x_corner_3, y_corner_3]
                corner_4 = [x_corner_4, y_corner_4]
                
                return corner_1, corner_2, corner_3, corner_4

In [14]:
def get_shifted_points(corner_1,corner_2,corner_3, corner_4, edge_scale_factor_1=0.2, edge_scale_factor_2 = 0.2, edge_scale_factor_3 = 0.2, edge_scale_factor_4 = 0.2):
    side_lengths = [
                    (distance(corner_1, corner_2), (corner_1, corner_2)),
                    (distance(corner_2, corner_3), (corner_2, corner_3)),
                    (distance(corner_3, corner_4), (corner_3, corner_4)),
                    (distance(corner_4, corner_1), (corner_4, corner_1)),
                ]

    #Box Stuff
    # Calculate the lengths of the sides

    # Sort sides by length in descending order and get the two longest sides
    longest_sides = sorted(side_lengths, key=lambda x: x[0], reverse=True)[:2]

    # Extract the lengths and coordinates of the two longest sides

    longest_sides_coords = [side[1] for side in longest_sides]
    
    x1 = longest_sides_coords[0][0][0]
    y1 = longest_sides_coords[0][0][1]
    
    x2 = longest_sides_coords[0][1][0]
    y2 = longest_sides_coords[0][1][1]
    
    x3 = longest_sides_coords[1][0][0]
    y3 = longest_sides_coords[1][0][1]
    
    x4 = longest_sides_coords[1][1][0]
    y4 = longest_sides_coords[1][1][1]

    
    p1, p2  = [x1,y1], [x2,y2]
    p3,p4 = [x3,y3], [x4,y4]
    

    d1 = calculate_distance(p1,p2)
    d2 = calculate_distance(p3,p4)
  
    p_new_1 = move_point_on_line(p1, p2, p1, d1 * edge_scale_factor_1)
    p_new_2 = move_point_on_line(p1, p2, p2, -d1* edge_scale_factor_2)
    p_new_3 = move_point_on_line(p3,p4, p3, d2*edge_scale_factor_3)
    p_new_4 = move_point_on_line(p3,p4,p4,d2*-edge_scale_factor_4 )
    
    return p_new_1, p_new_2, p_new_3, p_new_4, d1,d2, x1,y1,x2,y2,x3,y3,x4,y4

In [15]:
def get_scale_factors(corner_1, corner_2, corner_3, corner_4):
    
                m1,q1 = line_from_points(corner_1,corner_3)
                m2,q2 = line_from_points(corner_2,corner_4)
    
                center = get_line_intercection(m1,m2,q1,q2)
                #cv2.circle(color_image, center,20, (50, 255, 0), 5) 
                
                diagonal_distance_13 = calculate_distance(corner_1,corner_3)
                diagonal_distance_24 = calculate_distance(corner_2,corner_4)
                distance_1 = calculate_distance(corner_1,center)
                distance_2 = calculate_distance(corner_3,center)
                distance_3 = calculate_distance(corner_2,center)
                distance_4 = calculate_distance(corner_4,center)

                factor_1 = distance_1 / diagonal_distance_13 * 0.4
                factor_2 = distance_2 / diagonal_distance_13 * 0.4
                factor_3 = distance_3 / diagonal_distance_24 * 0.4
                factor_4 = distance_4 / diagonal_distance_24 * 0.4
                
                return factor_1, factor_2, factor_3, factor_4

In [16]:
Videocapture = False

if Videocapture == True:  
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Adjust codec as needed (e.g., 'XVID', 'MJPG', 'MP4V')
    out_color = cv2.VideoWriter('Demo/BinDetection_v1_models_.avi', fourcc,30.0, (1920, 1080))
# Check if the VideoWriter object is successfully initialized
    if not out_color.isOpened():
        print("Error: Could not open video writer.")



save_dir = 'batch2/negative'
os.makedirs(save_dir,exist_ok = True)
image_count = 0
Save_images = False

In [18]:
#Initialize
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, framerate = 30)
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, framerate = 30)

pipeline.start(config)

frame_queue = queue.Queue(maxsize=1) 
#create align object
align_to = rs.stream.color
align = rs.align(align_to)


capture_thread = threading.Thread(target=capture_frames, args = (pipeline,frame_queue,align), daemon=True)

capture_thread.start()


try:
    while True:
            if not frame_queue.empty():
        
                start = time.time()
                color_image, depth_image = frame_queue.get()
     
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.25), cv2.COLORMAP_JET)
                masked_color_image, hull,box= cut_region(depth_image,color_image,min_depth = 0,max_depth = 0.8)
                
                ####################################

                #get Hull corner
              
                corner_1, corner_2, corner_3, corner_4 = get_corner_points(color_image, box, hull)

                ############ draw rectangle end
                
                #get_scale_factor

                
                factor_1, factor_2, factor_3, factor_4 = get_scale_factors(corner_1, corner_2, corner_3, corner_4)

                
                #line1 = (corner_1,corner_3)
                #line2 = (corner_2,corner_4)
                
                #if line1:   
                   # cv2.line(color_image, line1[0], line1[1], (255, 255, 0), 5)
                #if line2: 
                   # cv2.line(color_image, line2[0], line2[1], (125, 0, 125), 5)
                    
                #######################################
                

                p_new_1, p_new_2, p_new_3, p_new_4, d1,d2, x1,y1,x2,y2,x3,y3,x4,y4 = get_shifted_points(corner_1,corner_2,corner_3, corner_4, factor_1, factor_2, factor_4, factor_3)

                ########## draw rectangle
      
                rect_1 = draw_rotated_rectangle(color_image,x1,x2,y1,y2,p_new_1)
                rect_2 = draw_rotated_rectangle(color_image,x1,x2,y1,y2,p_new_2)
                rect_3 = draw_rotated_rectangle(color_image,x3,x4,y3,y4,p_new_3)
                rect_4 = draw_rotated_rectangle(color_image,x3,x4,y3,y4,p_new_4)
                
   
                scale_factor = 0.8
                resized_image = cv2.resize(color_image,None,fx =  scale_factor,fy = scale_factor)
                cv2.imshow('masked image', resized_image)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
      

finally:
    # Stop streaming
    pipeline.stop()
        
    if Videocapture == True:
        out_color.release()
    cv2.destroyAllWindows()
    

C:\Users\nmamie\AppData\Local\Temp\ipykernel_10556\2748353546.py:25: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Users\nmamie\AppData\Local\anaconda3\envs\binsertion-env\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\nmamie\AppData\Local\anaconda3\envs\binsertion-env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\nmamie\AppData\Local\anaconda3\envs\binsertion-env\lib\threading.py", line 870, in run


KeyboardInterrupt: 

    self._target(*self._args, **self._kwargs)
  File "c:\Users\nmamie\VSCode\BinInsertion_main\video_capture.py", line 14, in capture_frames
    frames = pipeline.wait_for_frames()
RuntimeError: wait_for_frames cannot be called before start()


In [257]:
box

array([[ 414,  182],
       [1594,  165],
       [1606,  974],
       [ 426,  991]])